In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import itertools


import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline


from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf  
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm 

# Ignore warning
import warnings
warnings.filterwarnings('ignore')


In [ ]:
df = pd.concat(
    [   
        pd.read_csv(
            "../input/crimes-in-chicago/Chicago_Crimes_2001_to_2004.csv",
            error_bad_lines=False,
        ),
        
        pd.read_csv(
            "../input/crimes-in-chicago/Chicago_Crimes_2005_to_2007.csv",
            error_bad_lines=False,
        ),
        pd.read_csv(
            "../input/crimes-in-chicago/Chicago_Crimes_2008_to_2011.csv",
            error_bad_lines=False,
        ),
        pd.read_csv(
            "../input/crimes-in-chicago/Chicago_Crimes_2012_to_2017.csv",
            error_bad_lines=False,
        ),
    ]
)
df.dtypes

In [ ]:
df.isna().sum()

In [ ]:
df['Date'] =  pd.to_datetime(df['Date'], format='%m/%d/%Y %H:%M:%S %p')

In [ ]:
df['Primary Type'].value_counts()

In [ ]:
df = df[(df['Primary Type'] != 'PUBLIC PEACE VIOLATION')]
df = df[(df['Primary Type'] != 'WEAPONS VIOLATION')]
df = df[(df['Primary Type'] != 'INTERFERENCE WITH PUBLIC OFFICER')]
df = df[(df['Primary Type'] != 'LIQUOR LAW VIOLATION')]
df = df[(df['Primary Type'] != 'INTIMIDATION')]
df = df[(df['Primary Type'] != 'STALKING')]
df = df[(df['Primary Type'] != 'OBSCENITY')]
df = df[(df['Primary Type'] != 'CONCEALED CARRY LICENSE VIOLATION')]
df = df[(df['Primary Type'] != 'PUBLIC INDECENCY')]
df = df[(df['Primary Type'] != 'HUMAN TRAFFICKING')]
df = df[(df['Primary Type'] != 'RITUALISM')]
df = df[(df['Primary Type'] != 'NON-CRIMINAL (SUBJECT SPECIFIED)')]
df = df[(df['Primary Type'] != 'DOMESTIC VIOLENCE')]
df = df[(df['Primary Type'] != 'NON - CRIMINAL')]
df = df[(df['Primary Type'] != 'OTHER NARCOTIC VIOLATION')]
df = df[(df['Primary Type'] != 'KIDNAPPING')]
df = df[(df['Primary Type'] != 'HOMICIDE')]
df = df[(df['Primary Type'] != 'ARSON')]
df = df[(df['Primary Type'] != 'GAMBLING')]
df = df[(df['Primary Type'] != 'PROSTITUTION')]
df = df[(df['Primary Type'] != 'NON-CRIMINAL')]
df = df[(df['Primary Type'] != 'OTHER OFFENSE')]
df = df[(df['Primary Type'] != 'CRIMINAL TRESPASS')]
df = df[(df['Primary Type'] != 'OFFENSE INVOLVING CHILDREN')]
df = df[(df['Primary Type'] != 'CRIM SEXUAL ASSAULT')]
df = df[(df['Primary Type'] != 'SEX OFFENSE')]
df = df[(df['Primary Type'] != 'DECEPTIVE PRACTICE')]

In [ ]:
df.index = pd.DatetimeIndex(df['Date'])
df.sort_index(inplace=True)
df.info()

In [ ]:
df.tail()

In [ ]:
df = df[df['Date'] < '2017-01-19']

In [ ]:
df.head()

In [ ]:
monthly = pd.DataFrame(df[df['Primary Type']=='THEFT'].resample('M').size())
monthly.columns = ['THEFT']

for i in df['Primary Type'].unique():
    monthly[i] = pd.DataFrame(df[df['Primary Type']==i].resample('M').size()) 

# Create a total
monthly['Total'] = monthly.sum(axis=1)

# Create a list of the types

crimeTypes = monthly.columns

In [ ]:
monthly.head()

In [ ]:
monthly.info()

In [ ]:
monthly.plot(figsize=(15,6))
plt.show()

In [ ]:
# Initiate the figure and define size
plt.figure(1)
plt.figure(figsize=(15,20))

# Using a for loop to plot each type of crime with a 12 months-moving average
i = 521
for eachCrime in crimeTypes:    
    plt.subplot(i);
    monthly[eachCrime].plot(label='Total per month')
    monthly[eachCrime].rolling(window=12).mean().plot(color='red', linewidth=5, label=' Moving Average')
    plt.title(eachCrime, fontsize=12)
    plt.xlabel('')
    plt.legend(prop={'size':12})
    plt.tick_params(labelsize=12)
    i = i + 1

In [ ]:
plt.figure(1)
plt.figure(figsize=(15,20))

# Using a for loop to plot each type of crime with a moving average
i = 521
for eachCrime in crimeTypes:    
    plt.subplot(i);
    monthly[eachCrime].diff().plot(label='1st Difference')
    monthly[eachCrime].rolling(window=12).mean().diff().plot(color='red', linewidth=5, label=' Moving Average')
    plt.title(eachCrime, fontsize=12)
    plt.xlabel('')
    plt.legend(prop={'size':12})
    plt.tick_params(labelsize=12)
    i = i + 1

In [ ]:
# Create a data frame with the 1st difference
montlhy_1stDiff = monthly.diff().dropna()

# Create a data frame with the results of the ADF test for each type
adfullerResults = pd.DataFrame(columns=crimeTypes, index = ['ADF','Critical Value 1%','Critical Value 5%','p-value'])
for eachType in crimeTypes:
    adf = adfuller(montlhy_1stDiff[eachType])
    adfullerResults[eachType] = ("{:.4f}".format(adf[0]),                                 
                                 "{:.4f}".format(adf[4]['1%']),
                                 "{:.4f}".format(adf[4]['5%']),
                                 "{:.4f}".format(adf[1]),)                                
adfullerResults

In [ ]:
print(montlhy_1stDiff[crimeTypes[8]].name)
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(montlhy_1stDiff[crimeTypes[0]], lags=50, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(montlhy_1stDiff[crimeTypes[0]], lags=50, ax=ax2)

In [ ]:
# First, I'm creating a list of combinations of p,d,q
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, d and q triplets
pdq = list(itertools.product(p,d,q))

# Generate all different combinations of seasonal p, d and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
def myArima(crimeTypeSeries):
    '''
    Takes in a series, returns the result of the model.
    Steps:
    1 - Find the best parameters
    2 - Train the model
    3 - Plot diagnostics
    4 - Plot forecast
    '''
    
    print ('Training for:', crimeTypeSeries.name)
    
    # 1 - Find the best parameters for P, D, Q
    lowestAIC = 999999999
    bestOrder = 0
    bestSeasonal_order = 0
    i = 0 # progress indicator
    
    for param in pdq:    
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(crimeTypeSeries,
                                                order=param,
                                                seasonal_order=param_seasonal)
                results = mod.fit()                
                
                # Keep the parameters that gives the lowest AIC
                newAIC = results.aic                
                if newAIC < lowestAIC: 
                    lowestAIC = newAIC
                    bestOrder = param
                    bestSeasonal_order = param_seasonal
                i = i+1
                print('\r', 'Calculating: {:.0f}%'.format(i/64*100), end='')
                
            except:
                i = i+1
                print('\r', 'Calculating: {:.0f}%'.format(i/64*100), end='')
                continue
      
    # 2 - Train the model with the best parameters
    print('\nBest parameters: SARIMA{}x{}12'.format(param, param_seasonal))
    mod = sm.tsa.statespace.SARIMAX(crimeTypeSeries, 
                                    order=bestOrder, 
                                    seasonal_order=bestSeasonal_order)
    results = mod.fit()

    # Plot diagnostics
    results.plot_diagnostics(figsize=(15, 12))    
    plt.show()
    
    # Plot Forecast
    pred = results.get_prediction(start = 125, end = 160, dynamic=False)
    pred_ci = pred.conf_int()
    pred_ci.head()
    plt.figure(figsize=(15,6))
    ax = crimeTypeSeries.plot(label='observed')
    plt.title("Actual and Forecast", fontsize=12)
    pred.predicted_mean.plot(ax=ax, label=' forecast', color='red', alpha=.7, lw=4)
    ax.fill_between(pred_ci.index,
                    pred_ci.iloc[:, 0],
                    pred_ci.iloc[:, 1], color='r', alpha=.3)    
    ax.set_xlabel('Years')
    ax.set_ylabel('Number of crime')   
    plt.legend()
    plt.show()
    return results

In [ ]:
results = []
for i in crimeTypes:    
    results.append(myArima(monthly[i]))

In [ ]:
predictions = []
for i in range(0,len(results)):
    predictions.append(results[i].get_prediction(start = 125, end = 160, dynamic=False).predicted_mean)

In [ ]:
predictions = pd.DataFrame(predictions)
predictions = predictions.transpose()
predictions.columns = monthly.columns
errors = ((monthly - predictions)/monthly)
errors.dropna(how='all', inplace=True)
n = len(errors)

# Calculate the MAPE
MAPE = round((errors.abs().sum() / n)*100,2)
print ('The Mean Absolute Percentage Error for each type for the year 6/2011 to 05/2014')
print (MAPE)

In [ ]:
predictions_ci = []
for i in range(0,len(results)):
    predictions_ci.append(results[i].get_prediction(start = 125, end = 160, dynamic=False).conf_int())

In [ ]:
predictions_ci